Load sql extention, connect to database file and turn on foreign keys

In [1]:
%load_ext sql
%sql sqlite:///software_project_management.db
%sql PRAGMA foreign_keys = ON;

 * sqlite:///software_project_management.db
Done.


[]

### Drop all tables

In [2]:
%%sql
DROP TABLE IF EXISTS EmployeeActivity;
DROP TABLE IF EXISTS Activity;
DROP TABLE IF EXISTS Plan;
DROP TABLE IF EXISTS Project;
DROP TABLE IF EXISTS Employee;


 * sqlite:///software_project_management.db
Done.
Done.
Done.
Done.
Done.


[]

### Creates Employee table  
Constraints:
- employeeid is a primary key
- name and hourycost are not null
- hourly cost is a positive number

In [3]:
%%sql
CREATE TABLE IF NOT EXISTS Employee (
    employeeid INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    hourlycost FLOAT NOT NULL
    CHECK (hourlycost > 0)
);

 * sqlite:///software_project_management.db
Done.


[]

### Creates project table  
Constraints:
- projectid is a primary key
- no null values
- budget is a positive number
- leaderid is a foreign key referencing employeeid
- startdate is before enddate
- budget is more than 0


In [4]:
%%sql
CREATE TABLE IF NOT EXISTS Project (
    projectname TEXT PRIMARY KEY,
    leaderid INTEGER NOT NULL,
    budget FLOAT NOT NULL,
    startdate DATE NOT NULL,
    enddate DATE NOT NULL,
    FOREIGN KEY (leaderid) REFERENCES Employee(employeeid)
    CHECK (enddate > startdate AND budget > 0)
);

 * sqlite:///software_project_management.db
Done.


[]

### Creates Plan table  
Constraints:
- planname is a primary key
- no null values
- projectname is a foreign key referencing projectname (primary key in project table)
- startdate is before or the same as enddate (allowing one day plans)
- startdate and enddate are within the project start and end date (can be equal)

In [5]:
%%sql
CREATE TABLE IF NOT EXISTS Plan (
    planname TEXT PRIMARY KEY,
    projectname TEXT NOT NULL,
    startdate DATE NOT NULL,
    enddate DATE NOT NULL,
    FOREIGN KEY (projectname) REFERENCES Project(projectname)
    CHECK (startdate <= enddate)
);

CREATE TRIGGER IF NOT EXISTS check_plan_dates
BEFORE INSERT ON Plan
FOR EACH ROW
BEGIN
    SELECT RAISE(ABORT, 'Plan dates are not within project dates')
    FROM Project
    WHERE Project.projectname = NEW.projectname
    AND (NEW.startdate < Project.startdate OR NEW.enddate > Project.enddate);
END;

 * sqlite:///software_project_management.db
Done.
Done.


[]

### Creates Activity table
Constraints:
- activityid is a primary key
- no null values
- planname is a foreign key referencing planname (primary key in plan table)
- startdate is before or the same as enddate (allowing one day activities)
- startdate and enddate are within the plan start and end date (can be equal)

In [6]:
%%sql
CREATE TABLE IF NOT EXISTS Activity (
    activityid INTEGER PRIMARY KEY,
    planname TEXT NOT NULL,
    activitytype TEXT NOT NULL,
    startdate DATE NOT NULL,
    enddate DATE NOT NULL,
    FOREIGN KEY (planname) REFERENCES Plan(planname)
    CHECK (startdate <= enddate)
);

CREATE TRIGGER IF NOT EXISTS check_activity_dates
BEFORE INSERT ON Activity
FOR EACH ROW
BEGIN
    SELECT RAISE(ABORT, 'Activity dates are not within plan dates')
    FROM Plan
    WHERE Plan.planname = NEW.planname
    AND (NEW.startdate < Plan.startdate OR NEW.enddate > Plan.enddate);
END;

 * sqlite:///software_project_management.db
Done.
Done.


[]

### Creates EmployeeActivity table
Constraints:
- employeeid and activityid are a composite primary key
- employeeid is a foreign key referencing employeeid (primary key in employee table)
- activityid is a foreign key referencing activityid (primary key in activity table)
- no null values
- hours is a positive number

In [7]:
%%sql
CREATE TABLE IF NOT EXISTS EmployeeActivity (
    employeeid INTEGER NOT NULL,
    activityid INTEGER NOT NULL,
    hoursworked FLOAT NOT NULL,
    PRIMARY KEY (employeeid, activityid),
    FOREIGN KEY (employeeid) REFERENCES Employee(employeeid),
    FOREIGN KEY (activityid) REFERENCES Activity(activityid)
    CHECK (hoursworked > 0)
);

 * sqlite:///software_project_management.db
Done.


[]

### Add data to all tables
10 tuples in each table

In [ ]:
%%sql
-- 10 records for Employee table
INSERT INTO Employee (name, hourlycost) VALUES ('James Johnson', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Benjamin Dover', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('John Smith', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Jane Doe', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Eel On Musk', 250.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Amo Guss', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Oliver Klozoff', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Bea O Problem', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Anita Bath', 220.50);
INSERT INTO Employee (name, hourlycost) VALUES ('Ima Weiner ', 220.50);

-- 10 records for Project table
INSERT INTO Project VALUES ('A', 1, 10000.00, '2018-01-01', '2018-12-31');
INSERT INTO Project VALUES ('Fix twitter', 5, 200000.00, '2022-10-27', '2023-11-11');
INSERT INTO Project VALUES ('B', 3, 10000.00, '2012-01-01', '2014-12-31');
INSERT INTO Project VALUES ('Project 4', 4, 10000.00, '2015-01-01', '2017-12-31');
INSERT INTO Project VALUES ('Project 5', 5, 10000.00, '2020-01-01', '2023-12-31');
INSERT INTO Project VALUES ('Project 6', 1, 50000.00, '2022-11-23', '2023-12-31');
INSERT INTO Project VALUES ('Project 7', 5, 10000.00, '2019-01-01', '2019-12-31');
INSERT INTO Project VALUES ('Project 8', 3, 80000.00, '2022-12-21', '2023-05-22');
INSERT INTO Project VALUES ('Project 9', 4, 10000.00, '2020-10-21', '2022-12-31');
INSERT INTO Project VALUES ('Project 10', 5, 50000.00, '2018-05-01', '2019-12-31');

-- 10 records for Plan table
INSERT INTO Plan VALUES ('A', 'A', '2018-01-01', '2018-12-31');
INSERT INTO Plan VALUES ('B', 'Fix twitter', '2022-12-10', '2022-12-12');
INSERT INTO Plan VALUES ('Plan3', 'B', '2012-05-01', '2013-12-31');
INSERT INTO Plan VALUES ('Plan4', 'Project 4', '2015-03-01', '2016-12-31');
INSERT INTO Plan VALUES ('Plan5', 'Project 5', '2020-05-01', '2023-06-31');
INSERT INTO Plan VALUES ('Plan6', 'Project 6', '2022-12-23', '2023-01-31');
INSERT INTO Plan VALUES ('Plan7', 'Project 7', '2019-05-01', '2019-09-31');
INSERT INTO Plan VALUES ('Plan8', 'Project 8', '2023-01-21', '2023-04-22');
INSERT INTO Plan VALUES ('Plan9', 'Project 9', '2020-12-21', '2021-12-31');
INSERT INTO Plan VALUES ('Plan10', 'Project 10', '2018-12-01', '2019-02-31');

-- 10 records for Activity table
INSERT INTO Activity (planname, activitytype, startdate, enddate) VALUES ('A', 'requirement analysis', '2018-01-01', '2018-01-31');
INSERT INTO Activity (planname, activitytype, startdate, enddate) VALUES ('B', 'architecture design', '2022-12-10', '2022-12-12');
INSERT INTO Activity (planname, activitytype, startdate, enddate) VALUES ('Plan3', 'development', '2012-05-01', '2013-06-31');
INSERT INTO Activity (planname, activitytype, startdate, enddate) VALUES ('Plan4', 'testing', '2015-03-01', '2016-04-31');
INSERT INTO Activity (planname, activitytype, startdate, enddate) VALUES ('Plan5', 'deployment', '2020-05-02', '2020-06-31');
INSERT INTO Activity (planname, activitytype, startdate, enddate) VALUES ('Plan6', 'requirement analysis', '2022-12-24', '2023-01-20');
INSERT INTO Activity (planname, activitytype, startdate, enddate) VALUES ('Plan7', 'architecture design', '2019-05-02', '2019-06-30');
INSERT INTO Activity (planname, activitytype, startdate, enddate) VALUES ('Plan8', 'development', '2023-01-22', '2023-02-20');
INSERT INTO Activity (planname, activitytype, startdate, enddate) VALUES ('Plan9', 'testing', '2020-12-22', '2021-01-31');
INSERT INTO Activity (planname, activitytype, startdate, enddate) VALUES ('Plan10', 'deployment', '2018-12-02', '2018-12-31');

-- 10 records for EmployeeActivity table
INSERT INTO EmployeeActivity VALUES (1, 1, 10.5);
INSERT INTO EmployeeActivity VALUES (2, 2, 3);
INSERT INTO EmployeeActivity VALUES (3, 3, 2);
INSERT INTO EmployeeActivity VALUES (4, 4, 120);
INSERT INTO EmployeeActivity VALUES (5, 5, 30);
INSERT INTO EmployeeActivity VALUES (6, 6, 0.45);
INSERT INTO EmployeeActivity VALUES (7, 7, 70);
INSERT INTO EmployeeActivity VALUES (8, 8, 12);
INSERT INTO EmployeeActivity VALUES (9, 9, 50);
INSERT INTO EmployeeActivity VALUES (10, 10, 4);

### Database security
TODO write stuff here


*This does not work in sqllite, because users are not supported*

In [ ]:
#user creation (not supported in sqlite)
%%sql
CREATE USER 'manager'@'localhost' IDENTIFIED BY 'manager';
GRANT SELECT ON project.Employee TO 'manager'@'localhost';
GRANT SELECT ON project.Project TO 'manager'@'localhost';
GRANT SELECT ON project.Plan TO 'manager'@'localhost';
GRANT SELECT ON project.Activity TO 'manager'@'localhost';
GRANT SELECT ON project.EmployeeActivity TO 'manager'@'localhost';


CREATE USER 'Anitha Bath'@'localhost' IDENTIFIED BY 'Anitha Bath';
GRANT SELECT ON project.Project TO 'Anitha Bath'@'localhost';
GRANT SELECT ON project.Plan TO 'Anitha Bath'@'localhost';
GRANT SELECT ON project.Activity TO 'Anitha Bath'@'localhost';

CREATE USER 'Amo Guss'@'localhost' IDENTIFIED BY 'Amo Guss';
GRANT SELECT ON project.Project TO 'Amo Guss'@'localhost';
GRANT SELECT ON project.Plan TO 'Amo Guss'@'localhost';
GRANT SELECT ON project.Activity TO 'Amo Guss'@'localhost';

## Query questions

How many employees of for a project titled "A" are involved in its plan “B”?

In [10]:
%%sql
SELECT COUNT(*)
FROM EmployeeActivity
WHERE activityid IN (SELECT activityid FROM Activity WHERE planname = "B");

 * sqlite:///software_project_management.db
Done.


COUNT(*)
1


Retrieve the names of plans made for project “A” with least cost.

In [11]:
%%sql
WITH costsOfPlansInA AS
    (SELECT Plan.planname, SUM(EmployeeActivity.hoursworked * Employee.hourlycost) AS totalcost
    FROM Plan, Activity, EmployeeActivity, Employee
    WHERE Plan.planname = Activity.planname AND Activity.activityid = EmployeeActivity.activityid
    AND EmployeeActivity.employeeid = Employee.employeeid AND Plan.projectname = "A"
    GROUP BY Plan.planname),
leastCost AS
    (SELECT MIN(totalcost) AS mincost FROM costsOfPlansInA)

SELECT planname FROM costsOfPlansInA, leastCost
WHERE costsOfPlansInA.totalcost = leastCost.mincost;

 * sqlite:///software_project_management.db
Done.


planname
A


For each employee retrieve the name, project name and plan name with the most working time.

In [12]:
%%sql
WITH mostWorkingTime AS
    (SELECT employeeid, MAX(hoursworked) AS maxhours FROM EmployeeActivity GROUP BY employeeid)

SELECT Employee.name, Project.projectname, Plan.planname
FROM Employee, Project, Plan, Activity, EmployeeActivity, mostWorkingTime
WHERE Employee.employeeid = mostWorkingTime.employeeid AND Project.projectname = Plan.projectname
AND Plan.planname = Activity.planname AND Activity.activityid = EmployeeActivity.activityid
AND EmployeeActivity.employeeid = mostWorkingTime.employeeid
AND EmployeeActivity.hoursworked = mostWorkingTime.maxhours;

 * sqlite:///software_project_management.db
Done.


name,projectname,planname
James Johnson,A,A
Benjamin Dover,Fix twitter,B
John Smith,B,Plan3
Jane Doe,Project 4,Plan4
Eel On Musk,Project 5,Plan5
Amo Guss,Project 6,Plan6
Oliver Klozoff,Project 7,Plan7
Bea O Problem,Project 8,Plan8
Anita Bath,Project 9,Plan9
Ima Weiner,Project 10,Plan10


Retrieve all the employee’s name and their least working time with respect to different project.

In [13]:
%%sql
WITH leastWorkingTime AS
    (SELECT employeeid, projectname, MIN(hoursworked) AS minhours
    FROM EmployeeActivity, Activity, Plan
    WHERE EmployeeActivity.activityid = Activity.activityid AND Activity.planname = Plan.planname
    GROUP BY employeeid, projectname)
SELECT Employee.name, leastWorkingTime.projectname, leastWorkingTime.minhours
FROM Employee, leastWorkingTime
WHERE Employee.employeeid = leastWorkingTime.employeeid;

 * sqlite:///software_project_management.db
Done.


name,projectname,minhours
James Johnson,A,10.5
Benjamin Dover,Fix twitter,3.0
John Smith,B,2.0
Jane Doe,Project 4,120.0
Eel On Musk,Project 5,30.0
Amo Guss,Project 6,0.45
Oliver Klozoff,Project 7,70.0
Bea O Problem,Project 8,12.0
Anita Bath,Project 9,50.0
Ima Weiner,Project 10,4.0


Retrieve all the plans for project with order of their working period.

In [14]:
%%sql
SELECT *
FROM Plan
WHERE projectname = "A"
ORDER BY julianday(startdate);

 * sqlite:///software_project_management.db
Done.


planname,projectname,startdate,enddate
A,A,2018-01-01,2018-12-31
